In [4]:
pip install niapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires p

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization
import torch

In [5]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=[0] * X_train.shape[1], upper=[1] * X_train.shape[1])
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)


In [6]:
X_scaled = scaler.fit_transform(X_final)

In [7]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = np.ravel(y_encoded)
X_tensor = torch.FloatTensor(X_scaled.astype('float32'))
y_tensor = torch.LongTensor(y_encoded).ravel()


device = torch.device("cpu")
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [8]:
problem = SVMFeatureSelection(X_train, y_train)
task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)


selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(map(str, np.where(selected_features)[0])))

model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
print('Subset accuracy:', model_selected.score(X_test[:, selected_features], y_test))

model_all.fit(X_train, y_train)
print('All Features Accuracy:', model_all.score(X_test, y_test))

Number of selected features: 20
Selected features: 1, 3, 8, 9, 10, 13, 15, 18, 19, 21, 22, 26, 32, 33, 34, 37, 38, 40, 42, 45
Subset accuracy: 0.9247572815533981
All Features Accuracy: 0.920873786407767


In [9]:
X_final_nparray=np.array(X_final)
X_temparr=X_final_nparray[:,selected_features]
X_temp=pd.DataFrame(X_final_nparray[:,selected_features])
X_temp.head(10299)





,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.868773,-0.771213,-2.042964,1.485495,-0.945431,-3.891042,-0.371484,-0.499695,-0.945431,-0.411666,-0.706356,-0.706303,0.874510,-0.320117,-0.679412,-0.706356,-1.731883,0.230048,0.918871,-1.206949
1,-0.884263,-0.770620,-1.405613,1.269000,-0.929817,-1.772857,-0.349378,-0.506746,-0.929817,-0.411876,-0.726777,-0.706492,0.874510,-0.410863,-0.697827,-0.726777,-1.731546,-0.615071,0.908664,-1.122341
2,-0.876139,-0.769600,-0.967963,1.347501,-0.913598,-0.788590,-0.349378,-0.514226,-0.913598,-0.418336,-0.773143,-0.706402,0.864346,-0.450946,-0.702993,-0.773143,-1.731210,-2.456876,0.892681,-1.122341
3,-0.876139,-0.770205,0.358887,1.077732,-0.946015,-0.136505,-0.366495,-0.515988,-0.946015,-0.418336,-0.769766,-0.706428,0.864346,-0.316416,-0.703159,-0.769766,-1.730874,-2.228321,0.892681,-2.048469
4,-0.883152,-0.771156,0.935553,1.321739,-0.940846,-0.970273,-0.367818,-0.520570,-0.940846,-0.426723,-0.770272,-0.706489,0.866851,-0.395043,-0.704387,-0.770272,-1.730537,-1.305973,0.908067,-1.639764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10294,1.210539,0.384289,0.735092,-0.332102,0.738886,0.043661,-0.767326,-0.294744,0.738886,-0.731239,0.748778,0.326047,-0.593473,0.398894,0.647372,0.748778,1.730537,-0.127378,-0.477836,0.984414
10295,1.210539,0.856951,0.035068,-0.188236,1.076214,-0.522792,-0.756616,-0.302345,1.076214,-0.731239,0.714089,0.281338,-1.213350,0.291804,0.710708,0.714089,1.730874,-0.007074,-0.310328,0.735797
10296,0.880306,0.669623,-0.373917,0.044241,0.935694,-1.030175,-0.756616,-0.306312,0.935694,-0.718627,0.708206,0.212658,-1.213350,0.247496,0.531317,0.708206,1.731210,0.080272,-0.344355,0.884723
10297,0.880306,0.162166,0.050097,0.054367,0.560590,-1.325299,-0.694372,-0.286905,0.560590,-0.715810,0.681904,0.227469,-0.336173,-0.091354,0.673244,0.681904,1.731546,-0.539951,-0.470604,1.386772
